In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Graph data/NLP Seq2Seq DataSet 3 Final.csv')

In [4]:
import spacy
nlp_pipeline = spacy.load("en_core_web_sm")

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#from torchtext import data

In [6]:
import random
from tqdm import tqdm 
import re 
import unicodedata
from io import open
from __future__ import unicode_literals, print_function, division

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cpu')

In [9]:
MAX_VOCAB_SIZE = 30000
MIN_COUNT = 5
MAX_SEQUENCE_LENGTH = 20
BATCH_SIZE = 64


In [10]:
from torchtext.legacy import data


In [11]:
SOS_token = 0
EOS_token = 1

In [12]:
class Sequence:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [13]:
data_file = '/content/drive/MyDrive/Colab Notebooks/Graph data/NLP Seq2Seq DataSet 3 Final.csv'

In [14]:
def read_data():
    print("Reading lines...")

    # Read the file and split into lines
    df = pd.read_csv(data_file)

    # Split every line into pairs and normalize
    pairs = []
    for index, row in df.iterrows():
      pairs.append([row['lemmatized_question'], row['sequence']])

    input_sequence = Sequence("lemmatized_question")
    output_sequence = Sequence("sequence")

    return input_sequence, output_sequence, pairs

In [15]:
MAX_LENGTH = 50


In [16]:
def prepare_data():
  input_sequence, output_sequence, pairs = read_data()

  for pair in pairs:
    input_sequence.add_sentence(pair[0])
    output_sequence.add_sentence(pair[1])

  print("Counted words:")
  print(input_sequence.name, input_sequence.n_words)
  print(output_sequence.name, output_sequence.n_words)

  return input_sequence, output_sequence, pairs


In [17]:
input_sequence, output_sequence, pairs = prepare_data()
print(random.choice(pairs))


Reading lines...
Counted words:
lemmatized_question 163
sequence 121
['who {VERB 1} in branches that {VERB 2} in {LOCATION}', 'SUBJECT any | CONDITION any | RELATIONSHIP {VERB 1} | CONDITION any | SUBJECT Organization | CONDITION any | RELATIONSHIP {VERB 2} |CONDITION any | OBJECT Location | CONDITION name = {LOCATION}']


In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

  

In [19]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
def indexes_from_sentence(sequence, sentence):
    index_list = []
    for word in sentence.split(' '):
      if word in sequence.word2index:
        index_list.append(sequence.word2index[word])
      else:
        index_list.append(0)
    
    return index_list

    #return [sequence.word2index[word] for word in sentence.split(' ')]


def tensor_from_sentence(sequence, sentence):
    indexes = indexes_from_sentence(sequence, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_sequence, pair[0])
    target_tensor = tensor_from_sentence(output_sequence, pair[1])
    return (input_tensor, target_tensor)



In [21]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
  
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [22]:
import time
import math

def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [23]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [24]:
def train_iters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (time_since(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

In [25]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_sequence, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_sequence.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [26]:
def show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluate_and_show_attention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    #show_attention(input_sentence, output_words, attentions)

In [27]:
hidden_size = 256
encoder1 = EncoderRNN(input_sequence.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_sequence.n_words, dropout_p=0.1).to(device)

train_iters(encoder1, attn_decoder1, 15000, print_every=1000)


1m 5s (- 15m 22s) (1000 6%) 1.3746
2m 12s (- 14m 21s) (2000 13%) 0.5820
3m 20s (- 13m 20s) (3000 20%) 0.2654
4m 27s (- 12m 15s) (4000 26%) 0.1284
5m 38s (- 11m 16s) (5000 33%) 0.0773
6m 48s (- 10m 13s) (6000 40%) 0.0861
7m 58s (- 9m 6s) (7000 46%) 0.0709
9m 7s (- 7m 59s) (8000 53%) 0.0438
10m 17s (- 6m 51s) (9000 60%) 0.0268
11m 26s (- 5m 43s) (10000 66%) 0.0177
12m 36s (- 4m 35s) (11000 73%) 0.0233
13m 45s (- 3m 26s) (12000 80%) 0.0219
14m 53s (- 2m 17s) (13000 86%) 0.0115
16m 3s (- 1m 8s) (14000 93%) 0.0107
17m 13s (- 0m 0s) (15000 100%) 0.0084


In [33]:
evaluate_and_show_attention("who all did {VERB} {ORGANIZATION}")


input = who all did {VERB} {ORGANIZATION}
output = SUBJECT any | CONDITION any | RELATIONSHIP {VERB} | CONDITION any | OBJECT Organization | CONDITION name = {ORGANIZATION} <EOS>


In [29]:
evaluate_and_show_attention("who the did {VERB} {ORGANIZATION}")


input = who the did {VERB} {ORGANIZATION}
output = SUBJECT any | CONDITION any | RELATIONSHIP {VERB} | CONDITION any | OBJECT Organization | CONDITION name = {ORGANIZATION} <EOS>


In [30]:
evaluate_and_show_attention("what the hell happen with {ORGANIZATION} in the year 2018")

input = what the hell happen with {ORGANIZATION} in the year 2018
output = SUBJECT Organization | CONDITION name = {ORGANIZATION} | RELATIONSHIP any | CONDITION year = 2021 | OBJECT any <EOS>


In [31]:
evaluate_and_show_attention("what happen with {ORGANIZATION} in the year 2005")

input = what happen with {ORGANIZATION} in the year 2005
output = SUBJECT Organization | CONDITION name = {ORGANIZATION} | RELATIONSHIP any | CONDITION year = 2001 | OBJECT any <EOS>


In [40]:
evaluate_and_show_attention("how {PERSON 1} connected  {PERSON 2}")

input = how {PERSON 1} connected  {PERSON 2}
output = SUBJECT Person | CONDITION  name = {PERSON 1} | OPERATION SHORTEST_PATH | OBJECT Person | CONDITION name = {PERSON 2} <EOS>


In [48]:
evaluate_and_show_attention("is the person {PERSON 1} related to the person {PERSON 2}")

input = is the person {PERSON 1} related to the person {PERSON 2}
output = SUBJECT Person | CONDITION name = {PERSON 1} | OPERATION SHORTEST_PATH | OBJECT Person | CONDITION name = {PERSON 2} <EOS>


In [49]:


def evaluate_saved_model(input_sentence, encoder_saved, attn_decoder_saved):
    output_words, attentions = evaluate(
        encoder_saved, attn_decoder_saved, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))

In [50]:
torch.save(encoder1.state_dict(), 'encoder1-model.pt')

In [51]:
torch.save(attn_decoder1.state_dict(), 'attn_decoder1-model.pt')

In [52]:
hidden_size = 256
encoder_saved = EncoderRNN(input_sequence.n_words, hidden_size).to(device)
attn_decoder_saved = AttnDecoderRNN(hidden_size, output_sequence.n_words, dropout_p=0.1).to(device)

In [53]:
encoder_saved.load_state_dict(torch.load('encoder1-model.pt'))
attn_decoder_saved.load_state_dict(torch.load('attn_decoder1-model.pt'))

<All keys matched successfully>

In [54]:
evaluate_saved_model("is the person {PERSON 1} related to the person {PERSON 2}", encoder_saved, attn_decoder_saved)

input = is the person {PERSON 1} related to the person {PERSON 2}
output = SUBJECT Person | CONDITION name = {PERSON 1} | OPERATION SHORTEST_PATH | OBJECT Person | CONDITION name = {PERSON 2} <EOS>
